In [2]:
!pip install python-docx # docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=864777b09e1ddc42283c84c910deeebd0880333be6c8d2a677a6b42ad3f74410
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [3]:
# read docx file
import docx
from google.colab import files

# chinese tokenizer
import jieba
import jieba.analyse # keyword extraction
import jieba.posseg as pseg # part of speech tagging

In [4]:
import pandas as pd
import numpy as np
import re
from collections import Counter

In [37]:
uploaded = files.upload()
file_name = 'critic.docx'
doc = docx.Document(file_name)
doc = doc.tables

Saving critic.docx to critic.docx


In [38]:
doc[0].rows[1].cells[1].text

'中国'

In [39]:
r = doc[0].rows
len(r)

102

In [40]:
r[1].cells[1].text

'中国'

In [41]:
cell = r[2]
cell.cells[1].text

'下了两个单，眼霜漏发，商家找不找人'

In [42]:
word1 = []
word2 = []

In [43]:
for i in range(len(r)):
    cell = r[i]
    col1 = cell.cells[1].text
    word1.append(col1)
    #print(cell)
    #for j in range(len(r)):

In [44]:
print(word1)

['差评', '中国', '下了两个单，眼霜漏发，商家找不找人', '买的套装少了赠品就算了，主品也不发', '包装和图片不符，毫无档次感', '几百块的精华居然也是酒精味', '缺货了就别卖，里面空着一个算什么，像假货', '保质期3年，发的使用日期在2年内', '小瓶身有粉末也不知道是啥', '没有按图片发货，客服态度差', '商品一般，并且标签没有贴好，皱褶', '一收到打开就知道是假货啊。。。。包装是不一样的', '货物丢失，但显示签收一直踢皮球', '再也不会在这买了，日期陈旧，胡弄人呢', '互动买的 赠品竟然还 没有平时买怎品多 做活动啥意思', '客服处理问题不好 推荐有误 赠品不发货\n活动非常死板，真是愧于大品牌', '我没有收到赠品呀，咋回事儿', '这个包装真的是没谁了，套装还能混发', '太小了 退货了 退款速度也慢 也没有运费险 退货还得自己承担', '外包装盒子打开的。。', '七夕送的礼物，分两次发的，都过完七夕了发来有啥用', '味道不好闻，去完脸上偶尔会有红点', '正品没有寄过来，只寄了小样。', '刚买完就降价，保价有什么用。', '买了送一套盒子？有必要这么节约？', '什么设计，一倒半瓶没了\n', '礼盒比单买贵很多。自用还是单买划算', '商品附加配件与描述完全不符\n', '买的精华液，没发只发赠品\n', '使用过了效果不好包装一般', '货不对版，私自缺货，店大欺客', '购物体验不好，套路太多，晚买还多送5亳升防晒霜', '买了两套化妆品，就给一个包装盒，怎么送人？？？\n', '与宣传不符，完全没一点档次', '买贵了，还不知道真假，商场专柜50ml的都比这个便宜', '问什么原产地是英国，专柜买原产地是美国，是正品吗\n', '买了礼盒里面只有两个赠品，逗我呢\n', '化妆品选择实体购买，赠送加起来比实体还贵', '买三个套盒就给一个盒子，**❌一个，还是个又空又大的盒子', '商家不负责任，购买三套套装，放一起邮寄，乱不说，还少发东西\n', '用纸一包在盒子里散装，你让我怎么送得出手！', '节日时候提前把价格调上去，就等过了保价期在调整下来，哈哈。', '里面有个精华露没有里盖，动不动就弄撒', '东西来了一大堆，心情一下就不好了，送人的东西，也不说清楚', '实际收到的和页面宣传不一样，联系客服称

In [45]:
word1 = word1[2:]

In [46]:
len(word1)
print(word1)

['下了两个单，眼霜漏发，商家找不找人', '买的套装少了赠品就算了，主品也不发', '包装和图片不符，毫无档次感', '几百块的精华居然也是酒精味', '缺货了就别卖，里面空着一个算什么，像假货', '保质期3年，发的使用日期在2年内', '小瓶身有粉末也不知道是啥', '没有按图片发货，客服态度差', '商品一般，并且标签没有贴好，皱褶', '一收到打开就知道是假货啊。。。。包装是不一样的', '货物丢失，但显示签收一直踢皮球', '再也不会在这买了，日期陈旧，胡弄人呢', '互动买的 赠品竟然还 没有平时买怎品多 做活动啥意思', '客服处理问题不好 推荐有误 赠品不发货\n活动非常死板，真是愧于大品牌', '我没有收到赠品呀，咋回事儿', '这个包装真的是没谁了，套装还能混发', '太小了 退货了 退款速度也慢 也没有运费险 退货还得自己承担', '外包装盒子打开的。。', '七夕送的礼物，分两次发的，都过完七夕了发来有啥用', '味道不好闻，去完脸上偶尔会有红点', '正品没有寄过来，只寄了小样。', '刚买完就降价，保价有什么用。', '买了送一套盒子？有必要这么节约？', '什么设计，一倒半瓶没了\n', '礼盒比单买贵很多。自用还是单买划算', '商品附加配件与描述完全不符\n', '买的精华液，没发只发赠品\n', '使用过了效果不好包装一般', '货不对版，私自缺货，店大欺客', '购物体验不好，套路太多，晚买还多送5亳升防晒霜', '买了两套化妆品，就给一个包装盒，怎么送人？？？\n', '与宣传不符，完全没一点档次', '买贵了，还不知道真假，商场专柜50ml的都比这个便宜', '问什么原产地是英国，专柜买原产地是美国，是正品吗\n', '买了礼盒里面只有两个赠品，逗我呢\n', '化妆品选择实体购买，赠送加起来比实体还贵', '买三个套盒就给一个盒子，**❌一个，还是个又空又大的盒子', '商家不负责任，购买三套套装，放一起邮寄，乱不说，还少发东西\n', '用纸一包在盒子里散装，你让我怎么送得出手！', '节日时候提前把价格调上去，就等过了保价期在调整下来，哈哈。', '里面有个精华露没有里盖，动不动就弄撒', '东西来了一大堆，心情一下就不好了，送人的东西，也不说清楚', '实际收到的和页面宣传不一样，联系客服称活动改变了。那下单的时候

In [47]:
for i in range(len(r)):
    cell = r[i]
    col2 = cell.cells[2].text
    word2.append(col2)

print(word2, 10)
print(len(word2))

['差评', '韩国', '像附加的图片一样吧… 也不给赠品… 不在这里买了，呜呜', '虽然是坚持使用7、8年的产品，但是比在百货商店直接购买的产品抹几天，，就在c..线上商城购买的山寨产品一样的感觉，，皮肤感觉相似 我很失望。', '因为要送礼物，好不容易打来电话，说拜托你附上购物袋，但是哪里都没有，一个赠品也没有。 啊~ 心情特别不好 也不知道怎么送礼物的', '20岁初和30岁初使用的感觉好像不一样的。 确实上了年纪之后使用，感觉很好。 因为最近戴着口罩，长痘痘又长了，但使用这个就没事了。但是赠品好像是从哪里退货过来的。 其他的都放进箱子里了。但是没有这个。', '有奇怪的味道。抹完之后皮肤也会起痘 呜呜 不怎么样啊', '制造国不一样啊，本来是为了送礼才买的，但是不好意思，有点不舒服，有时间的话，打算去新世界百货商店卖场购买同样的商品。是为了确认制造国。 而且我购买的时候写着赠送商品券，但是一天之内就消失了。 购买各种不爽的礼物啊，又不是便宜货，真可惜。', '说实话，如果价格对比不太清楚。用过几瓶，皮肤完全看不出来变好，价格对比好像不怎么样。 这个也是Stady产品 说实话 用这个的话 韩国产品会更好一些', '就那样吧。', '用过的东西用完了,找的时候打折了很多,所以毫无疑义地买了两个,但是拆开使用的瞬间感觉不一样,该怎么表现呢?湿润?完全没有那种感觉,像涂了水一样的感觉,容器也不知不觉就不同了。 包装也是因为封面太松了，以为拿起来的瞬间会掉下来，吓了一跳。真心想确认是不是真品，剩下的一个想退款。', '没有效果.价格贵但是就那样吧。没有任何变化。花了很多钱买的，后悔了。听说还有假的，要小心了', '是因为心情所致吗？在别的地方用完小号之后，因为效果好，所以买了大号的，我使用的没有变化只有那个换。皮肤上总是长东西。 虽然不能变干净，但也不是闹着玩的，是心情所致吗？在别的地方买的地板上有保质期，但是没有这个吗？', '之前用过的没剩多少了，所以重新购买了。 新收到的产品是粉色，但颜色和香味完全不一样，使用感也不太好。 不知道该不该用，光这个精华已经用了20多年了，现在又长痘痘了，真是荒唐。', '太稀了，脸都紧绷绷的，便宜的再也不会买了。', '褐色瓶用了15多年，一直都是免税店百货商店网上购买的，价格低廉，所以买了，但是质感不一样... 黏糊糊的..

In [48]:
word2 = word2[2:]
len(word2)
print(word2)

['像附加的图片一样吧… 也不给赠品… 不在这里买了，呜呜', '虽然是坚持使用7、8年的产品，但是比在百货商店直接购买的产品抹几天，，就在c..线上商城购买的山寨产品一样的感觉，，皮肤感觉相似 我很失望。', '因为要送礼物，好不容易打来电话，说拜托你附上购物袋，但是哪里都没有，一个赠品也没有。 啊~ 心情特别不好 也不知道怎么送礼物的', '20岁初和30岁初使用的感觉好像不一样的。 确实上了年纪之后使用，感觉很好。 因为最近戴着口罩，长痘痘又长了，但使用这个就没事了。但是赠品好像是从哪里退货过来的。 其他的都放进箱子里了。但是没有这个。', '有奇怪的味道。抹完之后皮肤也会起痘 呜呜 不怎么样啊', '制造国不一样啊，本来是为了送礼才买的，但是不好意思，有点不舒服，有时间的话，打算去新世界百货商店卖场购买同样的商品。是为了确认制造国。 而且我购买的时候写着赠送商品券，但是一天之内就消失了。 购买各种不爽的礼物啊，又不是便宜货，真可惜。', '说实话，如果价格对比不太清楚。用过几瓶，皮肤完全看不出来变好，价格对比好像不怎么样。 这个也是Stady产品 说实话 用这个的话 韩国产品会更好一些', '就那样吧。', '用过的东西用完了,找的时候打折了很多,所以毫无疑义地买了两个,但是拆开使用的瞬间感觉不一样,该怎么表现呢?湿润?完全没有那种感觉,像涂了水一样的感觉,容器也不知不觉就不同了。 包装也是因为封面太松了，以为拿起来的瞬间会掉下来，吓了一跳。真心想确认是不是真品，剩下的一个想退款。', '没有效果.价格贵但是就那样吧。没有任何变化。花了很多钱买的，后悔了。听说还有假的，要小心了', '是因为心情所致吗？在别的地方用完小号之后，因为效果好，所以买了大号的，我使用的没有变化只有那个换。皮肤上总是长东西。 虽然不能变干净，但也不是闹着玩的，是心情所致吗？在别的地方买的地板上有保质期，但是没有这个吗？', '之前用过的没剩多少了，所以重新购买了。 新收到的产品是粉色，但颜色和香味完全不一样，使用感也不太好。 不知道该不该用，光这个精华已经用了20多年了，现在又长痘痘了，真是荒唐。', '太稀了，脸都紧绷绷的，便宜的再也不会买了。', '褐色瓶用了15多年，一直都是免税店百货商店网上购买的，价格低廉，所以买了，但是质感不一样... 黏糊糊的... 而且瓶子上还有英文字

In [49]:
data = pd.DataFrame({'chinese':word1, 'korean':word2})
print(data)

                                              chinese  \
0                                   下了两个单，眼霜漏发，商家找不找人   
1                                   买的套装少了赠品就算了，主品也不发   
2                                       包装和图片不符，毫无档次感   
3                                       几百块的精华居然也是酒精味   
4                                缺货了就别卖，里面空着一个算什么，像假货   
..                                                ...   
95  内包装实在是太丑了，里面这个毛样子送人的话实在是不太行，没一点档次，还赶不上几十块钱的路边摊...   
96  实物与图片无关，我是看着图片礼盒买的，结果收到的根本不一样，这样送人太尴尬了，联系客服一问三...   
97  很少给差评。收到精华液后陆续发现老有精华漏出到瓶外，刚开始以为是自己使用不小心漏了，后来仔细...   
98  同一天拍的一样的产品，赠送礼品竟然不一样，也是神奇～直播间说有的眼罩也没有，感觉很混乱，这是...   
99  昨天直播间下单送化妆包，今天收到产品没有赠送。联系客服页问，客服推诿，说联系不到直播间。真的...   

                                               korean  
0                         像附加的图片一样吧… 也不给赠品… 不在这里买了，呜呜  
1   虽然是坚持使用7、8年的产品，但是比在百货商店直接购买的产品抹几天，，就在c..线上商城购买...  
2   因为要送礼物，好不容易打来电话，说拜托你附上购物袋，但是哪里都没有，一个赠品也没有。 啊~ ...  
3   20岁初和30岁初使用的感觉好像不一样的。 确实上了年纪之后使用，感觉很好。 因为最近戴着口...  
4                          有奇怪的味道。抹

In [50]:
data.iloc[:,0]

0                                     下了两个单，眼霜漏发，商家找不找人
1                                     买的套装少了赠品就算了，主品也不发
2                                         包装和图片不符，毫无档次感
3                                         几百块的精华居然也是酒精味
4                                  缺货了就别卖，里面空着一个算什么，像假货
                            ...                        
95    内包装实在是太丑了，里面这个毛样子送人的话实在是不太行，没一点档次，还赶不上几十块钱的路边摊...
96    实物与图片无关，我是看着图片礼盒买的，结果收到的根本不一样，这样送人太尴尬了，联系客服一问三...
97    很少给差评。收到精华液后陆续发现老有精华漏出到瓶外，刚开始以为是自己使用不小心漏了，后来仔细...
98    同一天拍的一样的产品，赠送礼品竟然不一样，也是神奇～直播间说有的眼罩也没有，感觉很混乱，这是...
99    昨天直播间下单送化妆包，今天收到产品没有赠送。联系客服页问，客服推诿，说联系不到直播间。真的...
Name: chinese, Length: 100, dtype: object

In [51]:
chinese = ' '.join(data.chinese)
chinese = re.sub('[^\w]', ' ', chinese)
chinese[:]

'下了两个单 眼霜漏发 商家找不找人 买的套装少了赠品就算了 主品也不发 包装和图片不符 毫无档次感 几百块的精华居然也是酒精味 缺货了就别卖 里面空着一个算什么 像假货 保质期3年 发的使用日期在2年内 小瓶身有粉末也不知道是啥 没有按图片发货 客服态度差 商品一般 并且标签没有贴好 皱褶 一收到打开就知道是假货啊    包装是不一样的 货物丢失 但显示签收一直踢皮球 再也不会在这买了 日期陈旧 胡弄人呢 互动买的 赠品竟然还 没有平时买怎品多 做活动啥意思 客服处理问题不好 推荐有误 赠品不发货 活动非常死板 真是愧于大品牌 我没有收到赠品呀 咋回事儿 这个包装真的是没谁了 套装还能混发 太小了 退货了 退款速度也慢 也没有运费险 退货还得自己承担 外包装盒子打开的   七夕送的礼物 分两次发的 都过完七夕了发来有啥用 味道不好闻 去完脸上偶尔会有红点 正品没有寄过来 只寄了小样  刚买完就降价 保价有什么用  买了送一套盒子 有必要这么节约  什么设计 一倒半瓶没了  礼盒比单买贵很多 自用还是单买划算 商品附加配件与描述完全不符  买的精华液 没发只发赠品  使用过了效果不好包装一般 货不对版 私自缺货 店大欺客 购物体验不好 套路太多 晚买还多送5亳升防晒霜 买了两套化妆品 就给一个包装盒 怎么送人     与宣传不符 完全没一点档次 买贵了 还不知道真假 商场专柜50ml的都比这个便宜 问什么原产地是英国 专柜买原产地是美国 是正品吗  买了礼盒里面只有两个赠品 逗我呢  化妆品选择实体购买 赠送加起来比实体还贵 买三个套盒就给一个盒子    一个 还是个又空又大的盒子 商家不负责任 购买三套套装 放一起邮寄 乱不说 还少发东西  用纸一包在盒子里散装 你让我怎么送得出手  节日时候提前把价格调上去 就等过了保价期在调整下来 哈哈  里面有个精华露没有里盖 动不动就弄撒 东西来了一大堆 心情一下就不好了 送人的东西 也不说清楚 实际收到的和页面宣传不一样 联系客服称活动改变了 那下单的时候怎么不提前告知或者及时更改宣传呢 误导消费者  添加不了图片 所以发的视频 客服也没说出解决的方法 就干的不像话  买的雅诗兰黛 结果发了个水杯过来 出了售后期 京东店家都不管了  既然是礼盒套装 买的肯定是送礼了 能不能包装上用点心 里面是啥歪意儿  日期截止使用时间

In [52]:
#ex = jieba.lcut(chinese, cut_all = True)
chi_words = jieba.lcut(chinese, cut_all = True)

In [55]:
#words = [word for word in ex if word != ' ']
#print(words[:30])

chi_words = [word for word in chi_words if word != '']

In [56]:
#cnt = Counter(words)
#cnt.most_common(20) # mode

chi_cnt = Counter(chi_words)
chi_cnt.most_common(5)

[('了', 91), ('的', 75), ('买', 47), ('是', 41), ('也', 28)]

In [57]:
#print(cnt)
print(chi_cnt)

Counter({'了': 91, '的': 75, '买': 47, '是': 41, '也': 28, '包装': 25, '就': 24, '发': 20, '差': 20, '我': 20, '没有': 19, '礼盒': 18, '说': 18, '客服': 17, '送': 16, '不': 15, '一个': 15, '还': 15, '赠品': 14, '人': 13, '有': 13, '都': 13, '图片': 12, '用': 12, '给': 12, '京东': 12, '很': 12, '东西': 11, '快递': 11, '产品': 11, '收到': 10, '这': 10, '评': 10, '一次': 10, '什么': 9, '不知': 9, '知道': 9, '一样': 9, '不好': 9, '盒子': 9, '体验': 9, '怎么': 9, '套装': 8, '里面': 8, '在': 8, '你们': 8, '找': 7, '精华': 7, '假货': 7, '商品': 7, '非常': 7, '没': 7, '宣传': 7, '个': 7, '你': 7, '这样': 7, '就是': 7, '和': 6, '不符': 6, '不一': 6, '呢': 6, '问题': 6, '这个': 6, '七夕': 6, '只': 6, '还是': 6, '问': 6, '结果': 6, '吧': 6, '垃圾': 6, '使用': 5, '竟然': 5, '活动': 5, '真的': 5, '退货': 5, '自己': 5, '礼物': 5, '购物': 5, '两套': 5, '一点': 5, '专柜': 5, '联系': 5, '消费': 5, '售后': 5, '跟': 5, '直播': 5, '不到': 5, '太': 5, '第一': 5, '失望': 5, '漏': 4, '少': 4, '日期': 4, '啥': 4, '发货': 4, '一直': 4, '不会': 4, '弄': 4, '真是': 4, '大': 4, '脸上': 4, '刚': 4, '这么': 4, '贵': 4, '效果': 4, '化妆': 4, '盒': 4, '让': 4, '实际': 4, '消费者': 4, '雅诗兰黛': 

In [58]:
# Create the dataframe
chi_cnt_df = pd.DataFrame({
    'count': chi_cnt
})

# Pull a random 10 out
#cnt_df.sample(10)

In [59]:
chi_cnt_df.to_csv('chi_critic.csv', encoding='utf_8_sig')

In [60]:
korean = ' '.join(data.korean)
korean = re.sub('[^\w]', ' ', korean)

kor_words = jieba.lcut(korean, cut_all = True)
kor_words = [word for word in kor_words if word != '']
kor_words = [word for word in kor_words if word != ' ']

kor_cnt = Counter(kor_words)

print(kor_cnt)

kor_cnt_df = pd.DataFrame({
    'count': kor_cnt
})
kor_cnt_df.sample(10)

Counter({'的': 190, '了': 148, '是': 49, '没有': 46, '也': 43, '我': 43, '产品': 31, '用': 30, '一样': 29, '皮肤': 28, '所以': 25, '都': 24, '买': 23, '很': 23, '就': 21, '和': 21, '但': 21, '包装': 21, '啊': 20, '呜呜': 19, '因为': 19, '吗': 19, '但是': 18, '不一': 18, '购买': 17, '使用': 16, '不是': 16, '价格': 16, '在': 15, '好像': 15, '太': 15, '这个': 14, '真的': 14, '你们': 14, '感觉': 13, '便宜': 13, '精华': 13, '失望': 12, '礼物': 12, '商品': 12, '正品': 12, '什么': 12, '不知': 11, '痘痘': 11, '保质': 11, '保质期': 11, '精华液': 11, '订购': 11, '给': 10, '赠品': 10, '送礼': 10, '知道': 10, '怎么': 10, '有': 10, '这': 10, '后': 10, '像': 9, '吧': 9, '不': 9, '上': 9, '送礼物': 9, '又': 9, '脸': 9, '状态': 9, '要': 8, '的话': 8, '确认': 8, '香味': 8, '适合': 8, '一个': 7, '不好': 7, '好': 7, '着': 7, '长痘痘': 7, '味道': 7, '完': 7, '而且': 7, '过': 7, '完全': 7, '涂': 7, '对': 7, '人': 7, '配送': 7, '只': 7, '回复': 7, '说': 6, '其他': 6, '不怎么': 6, '有点': 6, '时候': 6, '说实话': 6, '实话': 6, '很多': 6, '是不是': 6, '想': 6, '不适': 6, '以前': 6, '个': 6, '来': 6, '红肿': 6, '虽然': 5, '年': 5, '百货': 5, '心情': 5, '长': 5, '像是': 5, '退货': 5, '会':

,count
按照,1
发烧,1
太过分,1
真糟糕,1
在意,1
而,1
包装,21
意思,1
才,4
更,5


In [61]:
kor_cnt_df.to_csv('kor_critic.csv', encoding='utf_8_sig')